In [1]:
from datasets import load_dataset

#spec_dataset = load_dataset("lewtun/github-issues", split="train")
spec_dataset = load_dataset('json', data_files='/home/jhdavis/repos/llms-for-hpc/prompts/omp-context.json', split="train")
spec_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['section_title', 'chunk'],
    num_rows: 1846
})

In [2]:
spec_dataset = spec_dataset.filter(lambda x: len(x["chunk"]) > 0)
spec_dataset

Filter:   0%|          | 0/1846 [00:00<?, ? examples/s]

Dataset({
    features: ['section_title', 'chunk'],
    num_rows: 1832
})

In [3]:
spec_dataset = spec_dataset.map(
    lambda x: {"text_length": len(x["chunk"].split())}
)

Map:   0%|          | 0/1832 [00:00<?, ? examples/s]

In [4]:
spec_dataset = spec_dataset.filter(lambda x: x["text_length"] > 20)
spec_dataset

Filter:   0%|          | 0/1832 [00:00<?, ? examples/s]

Dataset({
    features: ['section_title', 'chunk', 'text_length'],
    num_rows: 1789
})

In [5]:
def concatenate_text(examples):
    return {
        "text": examples["section_title"]
        + " \n "
        + examples["chunk"]
    }


spec_dataset = spec_dataset.map(concatenate_text)
spec_dataset

Map:   0%|          | 0/1789 [00:00<?, ? examples/s]

Dataset({
    features: ['section_title', 'chunk', 'text_length', 'text'],
    num_rows: 1789
})

In [6]:
import tensorflow as tf
tf.__version__

2024-02-23 15:00:10.962009: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 15:00:10.962055: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 15:00:10.966139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 15:00:10.975427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-23 15:00:12.010445: W tensorflow/compiler/tf2

'2.15.0'

In [7]:
import importlib.util
importlib.util.find_spec("tensorflow")

from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

2024-02-23 15:00:15.180476: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 15:00:15.182941: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-23 15:00:15.183112: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [9]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [10]:
embedding = get_embeddings(spec_dataset["text"][0])
embedding.shape

TensorShape([1, 768])

In [11]:
embeddings_dataset = spec_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)

/home/jhdavis/.local/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Map:   0%|          | 0/1789 [00:00<?, ? examples/s]

In [12]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['section_title', 'chunk', 'text_length', 'text', 'embeddings'],
    num_rows: 1789
})

In [21]:
question = "Scan problem: Replace the i-th element of the vector x with the minimum value from indices 0 through i. Use OpenMP to compute in parallel."
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [22]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [23]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [24]:
for _, row in samples_df.iterrows():
    print(f"SPEC CHUNK: {row.chunk}")
    print(f"SECTION: {row.section_title}")
    print(f"SCORE: {row.scores}")
    print("=" * 50)
    print()

SPEC CHUNK: 
 • The memspace-handle argument for the mem-space modifier must be an identifier that matches 
 one of the predefined memory space names.
 Cross References 
 • Memory Allocators, see Section 6.2 
 • Memory Spaces, see Section 6.1 
 • omp_destroy_allocator, see Section 18.13.3 
 • omp_init_allocator, see Section 18.13.2 
 • target directive, see Section 13.8 
 OpenMP API – Version 5.2 November 2021 

SECTION: 6.8 uses_allocators Clause
SCORE: 36.951053619384766

SPEC CHUNK: RUNTIME LIBRARY ROUTINES 423 
 integer(kind=omp_alloctrait_val_kind), & 
 parameter :: omp_atv_serialized = 5 
 integer(kind=omp_alloctrait_val_kind), & 
 parameter :: omp_atv_sequential = & 
 omp_atv_serialized ! (deprecated) 
 integer(kind=omp_alloctrait_val_kind), & 
 parameter :: omp_atv_private = 6 
 integer(kind=omp_alloctrait_val_kind), & 
 parameter :: omp_atv_all = 7 
 integer(kind=omp_alloctrait_val_kind), & 
 parameter :: omp_atv_thread = 8 
 integer(kind=omp_alloctrait_val_kind), & 
 paramete